In [47]:
from calculate_level_new import get_level_from_raw_text

In [2]:
import psycopg2

In [3]:
conn = psycopg2.connect(dbname='pgstage', user='linguist', 
                        password='eDQGK0GCStlYlHNV', host='192.168.122.183')

In [4]:
cursor = conn.cursor()

In [8]:
conn.rollback()
cursor.execute("SELECT jdesc ->>'page_text' AS page_text, jungle_id FROM public.content_jungle_pages LIMIT 20")
cursor.fetchone() 

("What I thought I would talk about today is the transition from one mode of thinking about nature to another that's tracked by architecture.  What's interesting about architects is, we always have tried to justify beauty by looking to nature, and arguably, beautiful architecture has always been looking at a model of nature.\r\n\r\n So, for roughly 300 years, the hot debate in architecture was whether the number five or the number seven was a better proportion to think about architecture, because the nose was one-fifth of your head, or because your head was one-seventh of your body.  And the reason that that was the model of beauty and of nature was because the decimal point had not been invented yet -- it was in the 16th century -- and everybody had to dimension a building in terms of fractions, so a room would be dimensioned as one-fourth of a facade;  the structural dais of that might be dimensioned as 10 units, and you would get down to the small elements by fractional subdivision:

In [51]:
import json

In [54]:
def write_response (json_file, start_index, final_index):
    file_name = './check_results/' + str(start_index) + '-' + str(final_index) +'.json'
    print("\nNOW SAVING", file_name,'\n')
    with open(file_name, 'w', encoding = "utf-8") as outfile:
        json.dump(json_file, outfile, indent=4, separators=(',', ':'),ensure_ascii=False)

In [ ]:
conn.rollback()
cursor.execute("SELECT jdesc ->>'page_text' AS page_text, jungle_id FROM public.content_jungle_pages")
level_json = []
current_text = {"text": '', 'jungle_id':0}
check_index = 0
start_index = 0
for row in cursor:
    if(row[1] != current_text['jungle_id']):
        if len(current_text['text']) > 0:
            print("====NEW TEXT (CALCULATE PREVIOUS)====\n")
            level  = get_level_from_raw_text(current_text['text'])
            level_json.append({"jungle_id": current_text['jungle_id'], "level":level})
            current_text = {"text": row[0], 'jungle_id':row[1]}
        else:
            print("====FIRST ENTRY====\n")
            current_text['jungle_id'] = row[1]
            current_text['text'] += ' ' + row[0]
    else:
        print("====ADD TEXT TO EXISTING====\n")
        current_text['text'] += ' ' + row[0]
    print(current_text)     
    print("===NEW ROW==\n")
    
    check_index += 1
    if check_index % 250 == 0 and check_index != 0:
        write_response(level_json,start_index, check_index )
        level_json = []
        start_index = check_index
if check_index!= start_index:
    write_response(level_json,start_index, check_index )
#print(level_json)